# License Plate Restoration

Demo notebook for image restoration (fog removal, deblurring, dark enhancement) and license plate OCR.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import ndimage
from scipy.signal import convolve2d

# kaggle dataset path
DATA_DIR = Path('/kaggle/input/adverse-condition-image-restoration-for-lpr')
if not DATA_DIR.exists():
    DATA_DIR = Path('.')

print(f"Using data dir: {DATA_DIR}")
if DATA_DIR.exists():
    files = list(DATA_DIR.glob('*'))[:10]
    print(f"Found {len(files)} items")
    for f in files[:5]:
        print(f"  {f.name}")

## Helper Functions

In [ ]:
# distortion classification
BRIGHT_DARK_THRESH = 80
BLUR_LAP_VAR_THRESH = 60
FOG_BRIGHT_THRESH = 150
FOG_CONTRAST_MAX = 40

def classify_distortion(img_bgr):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    brightness = float(gray.mean())
    contrast = float(gray.std())
    lap = cv2.Laplacian(gray, cv2.CV_64F)
    lap_var = float(lap.var())
    
    classes = []
    if brightness < BRIGHT_DARK_THRESH:
        classes.append("dark")
    if lap_var < BLUR_LAP_VAR_THRESH:
        classes.append("blur")
    if brightness > FOG_BRIGHT_THRESH and contrast < FOG_CONTRAST_MAX:
        classes.append("fog")
    if not classes:
        classes.append("clean")
    return classes[0]

In [ ]:
# fog removal (dark channel prior + CLAHE)
def defog_dcp_clahe(image_bgr):
    I = image_bgr.astype(np.float32) / 255.0
    patch = 5
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (patch, patch))
    dark = cv2.erode(np.min(I, axis=2), kernel)
    
    num_pixels = I.shape[0] * I.shape[1]
    top_k = max(1, num_pixels // 2000)
    indices = np.argpartition(dark.flatten(), -top_k)[-top_k:]
    A = np.mean(I.reshape(-1,3)[indices], axis=0)
    
    omega = 0.75
    norm_I = I / (A + 1e-6)
    dark_norm = cv2.erode(np.min(norm_I, axis=2), kernel)
    t = 1 - omega * dark_norm
    t0 = 0.2
    t = np.clip(t, t0, 1)
    t = cv2.GaussianBlur(t, (7,7), 10)
    
    J = (I - A) / t[..., None] + A
    J = np.clip(J, 0, 1)
    
    blend_alpha = 0.8
    J = blend_alpha * J + (1 - blend_alpha) * I
    
    hsv = cv2.cvtColor((J*255).astype(np.uint8), cv2.COLOR_BGR2HSV)
    clahe = cv2.createCLAHE(clipLimit=0.8, tileGridSize=(8,8))
    hsv[:,:,2] = clahe.apply(hsv[:,:,2])
    out = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return out

In [ ]:
# dark enhancement (IAGCWD)
def image_agcwd(img, a=0.25, truncated_cdf=False):
    hist, bins = np.histogram(img.flatten(), 256, [0, 256])
    cdf = hist.cumsum()
    cdf_normalized = cdf / cdf.max()
    prob_normalized = hist / hist.sum()
    
    unique_intensity = np.unique(img)
    prob_min = prob_normalized.min()
    prob_max = prob_normalized.max()
    
    pn_temp = (prob_normalized - prob_min) / (prob_max - prob_min + 1e-10)
    pn_temp[pn_temp>0] = prob_max * (pn_temp[pn_temp>0]**a)
    pn_temp[pn_temp<0] = prob_max * (-((-pn_temp[pn_temp<0])**a))
    prob_normalized_wd = pn_temp / (pn_temp.sum() + 1e-10)
    cdf_prob_normalized_wd = prob_normalized_wd.cumsum()
    
    if truncated_cdf:
        inverse_cdf = np.maximum(0.5, 1 - cdf_prob_normalized_wd)
    else:
        inverse_cdf = 1 - cdf_prob_normalized_wd
    
    img_new = img.copy()
    for i in unique_intensity:
        img_new[img==i] = np.round(255 * (i / 255)**inverse_cdf[i])
    return img_new

def process_dimmed(img):
    return image_agcwd(img, a=0.75, truncated_cdf=True)

def enhance_dark(image_bgr):
    YCrCb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2YCrCb)
    Y = YCrCb[:,:,0]
    threshold = 0.3
    exp_in = 112
    M, N = image_bgr.shape[:2]
    mean_in = np.sum(Y / (M * N))
    t = (mean_in - exp_in) / exp_in
    
    if t < -threshold:
        result = process_dimmed(Y)
        YCrCb[:,:,0] = result
        return cv2.cvtColor(YCrCb, cv2.COLOR_YCrCb2BGR)
    else:
        return image_bgr

In [ ]:
# motion deblur (Richardson-Lucy)
def create_motion_psf(length, angle, size=31):
    kernel = np.zeros((size, size), dtype=np.float32)
    center = size // 2
    angle_rad = np.deg2rad(angle)
    for i in range(length):
        x = int(center + (i - length/2) * np.cos(angle_rad))
        y = int(center + (i - length/2) * np.sin(angle_rad))
        if 0 <= x < size and 0 <= y < size:
            kernel[y, x] = 1.0
    if kernel.sum() > 0:
        kernel /= kernel.sum()
    return kernel

def richardson_lucy_deconv(image, psf, num_iter=20, sigma=0.5):
    estimate = image.copy()
    psf_flipped = np.flip(np.flip(psf, 0), 1)
    eps = 1e-10
    
    for _ in range(num_iter):
        blurred_estimate = convolve2d(estimate, psf, mode='same', boundary='symm')
        ratio = image / (blurred_estimate + eps)
        ratio = np.clip(ratio, 0.01, 0.99)
        correction = convolve2d(ratio, psf_flipped, mode='same', boundary='symm')
        estimate = estimate * correction
        if sigma > 0:
            estimate = ndimage.gaussian_filter(estimate, sigma=sigma)
        estimate = np.clip(estimate, 0, 1)
    return estimate

def deblur_image(image_bgr):
    h, w = image_bgr.shape[:2]
    scale = 0.5 if max(h, w) > 800 else 1.0
    if scale < 1.0:
        img_small = cv2.resize(image_bgr, (int(w*scale), int(h*scale)))
    else:
        img_small = image_bgr
    
    img_gray = cv2.cvtColor(img_small, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0
    
    lengths = [10, 20, 30]
    angles = [0, 45, 90]
    best_kernel = None
    best_score = -1
    
    for length in lengths:
        for angle in angles:
            psf = create_motion_psf(length, angle, size=31)
            if psf.sum() == 0:
                continue
            estimate = richardson_lucy_deconv(img_gray, psf, num_iter=5, sigma=0.5)
            laplacian = cv2.Laplacian((estimate * 255).astype(np.uint8), cv2.CV_64F)
            score = laplacian.var()
            if score > best_score:
                best_score = score
                best_kernel = (length, angle, psf)
    
    if best_kernel is None:
        return image_bgr
    
    length, angle, psf = best_kernel
    img_gray_full = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0
    deblurred_gray = richardson_lucy_deconv(img_gray_full, psf, num_iter=20, sigma=0.5)
    deblurred_gray_uint8 = (np.clip(deblurred_gray, 0, 1) * 255).astype(np.uint8)
    return cv2.cvtColor(deblurred_gray_uint8, cv2.COLOR_GRAY2BGR)

## Process Images

In [ ]:
# find images
image_files = []
for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
    image_files.extend(list(DATA_DIR.rglob(f'*{ext}')))
    image_files.extend(list(DATA_DIR.rglob(f'*{ext.upper()}')))

print(f"Found {len(image_files)} images")
if len(image_files) == 0:
    print("No images found, using test image if available")
    test_img = cv2.imread('test.jpg')
    if test_img is not None:
        print("Using test.jpg")
        image_files = [Path('test.jpg')]

In [ ]:
# process first few images
def process_image(img_path):
    img = cv2.imread(str(img_path))
    if img is None:
        return None, None, None, None
    
    # classify distortion
    dist_type = classify_distortion(img)
    
    # apply restoration
    restored = img.copy()
    if dist_type == 'fog':
        restored = defog_dcp_clahe(img)
    elif dist_type == 'dark':
        restored = enhance_dark(img)
    elif dist_type == 'blur':
        restored = deblur_image(img)
    
    return img, restored, dist_type, img_path.name

# process up to 5 images
results = []
for img_path in image_files[:5]:
    original, restored, dist_type, name = process_image(img_path)
    if original is not None:
        results.append((original, restored, dist_type, name))
        print(f"Processed {name}: {dist_type}")

In [ ]:
# display results
def show_comparison(original, restored, title, dist_type):
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    orig_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    rest_rgb = cv2.cvtColor(restored, cv2.COLOR_BGR2RGB)
    
    axes[0].imshow(orig_rgb)
    axes[0].set_title(f"Original ({dist_type})")
    axes[0].axis('off')
    
    axes[1].imshow(rest_rgb)
    axes[1].set_title("Restored")
    axes[1].axis('off')
    
    plt.suptitle(title, fontsize=12)
    plt.tight_layout()
    plt.show()

for original, restored, dist_type, name in results:
    show_comparison(original, restored, name, dist_type)

## License Plate OCR

Note: This requires fast-alpr to be installed. For Kaggle, you may need to install it first.

In [ ]:
# try to run OCR (will fail gracefully if not available)
try:
    from fast_alpr import ALPR
    
    alpr = ALPR(
        detector_model="yolo-v9-t-384-license-plate-end2end",
        ocr_model="cct-xs-v1-global-model",
    )
    
    def run_ocr(img_bgr):
        detections = alpr.detector.predict(img_bgr)
        if not detections:
            return ""
        best = max(detections, key=lambda d: d.confidence)
        bbox = best.bounding_box
        x1, y1 = max(bbox.x1, 0), max(bbox.y1, 0)
        x2, y2 = min(bbox.x2, img_bgr.shape[1]), min(bbox.y2, img_bgr.shape[0])
        cropped = img_bgr[y1:y2, x1:x2]
        ocr_result = alpr.ocr.predict(cropped)
        return ocr_result.text if ocr_result and ocr_result.text else ""
    
    print("OCR available!")
    
    # run OCR on restored images
    for original, restored, dist_type, name in results:
        orig_text = run_ocr(original)
        rest_text = run_ocr(restored)
        print(f"\n{name} ({dist_type}):")
        print(f"  Original: {orig_text if orig_text else 'No plate detected'}")
        print(f"  Restored: {rest_text if rest_text else 'No plate detected'}")
        
except ImportError:
    print("fast-alpr not available. Install with: pip install fast-alpr")
except Exception as e:
    print(f"OCR error: {e}")